# Tutorial: Loading an USBMD data file
In this tutorial notebook we will show how to load an USBMD data file and how to access the data stored in it.

In [ ]:
# Set the working directory to the root of the repository
# We do this by moving up until we find the file setup.cfg (which is in the
# root of the repository)
import os

while not os.path.exists('setup.cfg'):
    os.chdir('..')

In [ ]:
# This cell is tagged with "parameters" so that it can be injected into the
# notebook when it is run by the unit test testing this notebook.
# The variable quick mode will then be set to True, allowing the notebook to
# run quickly for testing purposes.
# injected parameters
quick_mode = False

In [ ]:
# imports
from usbmd.utils import log
from pathlib import Path

import h5py
import matplotlib.pyplot as plt
import numpy as np

from usbmd.datapaths import set_data_paths
from usbmd.data import load_usbmd_file
from usbmd.processing import log_compress, rf2iq, channels_to_complex
from usbmd.setup_usbmd import setup_config
from usbmd.utils.read_h5 import print_hdf5_attrs

## Viewing the file structure
The USBMD data format works with HDF5 files. We can open a USBMD data file using the `h5py` package and have a look at the contents using the print_hdf5_attrs function. You can see that every dataset element contains a corresponding description and unit.

We will demonstrate this by loading some example cardiac data from the NAS. We assume that the `Ultrasound-BMD` folder on the NAS is mapped to `Z:/` on your computer.

> *Tip:*
> You can also use the [HDFView](https://www.hdfgroup.org/downloads/hdfview/) tool to view the contents of the USBMD data file without having to run any code.

In [ ]:
# Load the config file
config_path = Path('configs', 'config_usbmd_cirs.yaml')
config = setup_config(config_path)

# Define path to the data file
data_root = set_data_paths("./users.yaml")['data_root']

# Plane wave data
data_path = data_root / config.data.dataset_folder / config.data.file_path

# Uncomment to use the linescanning data instead
# data_path = data_root / r'USBMD-example-data/linescanning_l115v.hdf5'

if not quick_mode:
    # Open the file and print the contents
    with h5py.File(data_path, 'r') as f:
        print_hdf5_attrs(f)
else:
    # If quick mode generate a dummy data file. This is mainly for
    # running the notebook on the github server.
    from usbmd.data import generate_example_dataset
    data_path = r"generated_dummy_data.hdf5"
    generate_example_dataset(data_path, add_optional_fields=True)
    log.warning(f"Data file not found. Generated dummy data file.")

## Loading the file with the toolbox

In [ ]:
# Load the data file and construct a probe and scan object
# We will only load the first two frames of the data file
selected_frames = [0, 1]
data, scan, probe = load_usbmd_file(data_path, frames=selected_frames, config=config)

# Print some info about the data
print('Data file loaded successfully')
print('The data tensor has shape: {}'.format(data.shape))
print('The dimensions of the data are (n_frames, n_transmits, n_axial_samples, '
      'n_elements, n_rf_iq_channels)')

# Reduce the beamforming grid in quick mode (during unit testing)
if quick_mode:
    scan.Nx = 32
    scan.Nz = 32

## Beamforming the data
Now we would like to beamform the data to generate an image. For this we need to load a config file that contains the beamforming parameters. We can load a config using the `load_config_from_yaml` function. Once we have a config we create the beamformer using get_beamformer with the scan and probe class that we got from the data file.

In [ ]:
# Lets import the correct ML libraries
if config.ml_library == 'torch':
    import torch
    from usbmd.backend.pytorch.layers.beamformers import get_beamformer
elif config.ml_library == 'tensorflow':
    import tensorflow as tf
    from usbmd.backend.tensorflow.layers.beamformers import get_beamformer

# Create the beamformer
config.model.batch_size = len(selected_frames)
beamformer = get_beamformer(probe=probe, scan=scan, config=config)

# # Turn the data into a torch tensor if we are using pytorch
if config.ml_library == 'torch':
    data = torch.from_numpy(data)

beamformed_data = beamformer(data)

# Convert the data to a numpy array and remove dummy dimensions
beamformed_data = beamformed_data.numpy()[0, ..., 0]

# Transform the data to IQ data
iq_data = rf2iq(
    beamformed_data,
    fs=scan.fs,
    fc=scan.fc,
    bandwidth=scan.bandwidth_percent,
    separate_channels=False)

# Take the magnitude of the complex data (the envelope)
image = np.abs(iq_data)

# Normalize the image and apply log compression
image = image/image.max()
image = log_compress(image)

## Plotting the result

In [ ]:
# Plot the image
fig, ax = plt.subplots(figsize=(6, 6))
ax.imshow(image,
          cmap='gray',
          extent=[scan.xlims[0], scan.xlims[1], scan.zlims[1], scan.zlims[0]],
          vmin=config.data.dynamic_range[0], vmax=config.data.dynamic_range[1])

# Add labels
ax.set_xlabel('Lateral distance [m]')
ax.set_ylabel('Depth [m]')

# Turn the figure black
ax.set_facecolor((0, 0, 0))
fig.patch.set_facecolor((0, 0, 0))

# Turn the ticks white
ax.tick_params(axis='both', colors='white')

# Turn the labels white
ax.xaxis.label.set_color('white')
ax.yaxis.label.set_color('white')

## Comparing to the image stored in the data file
Since this data file also already has processed image data inside we can plot this too and see if the result is the same.

In [ ]:
# Load the image data from the data file
try:
    image = load_usbmd_file(data_path, frames=selected_frames, data_type="beamformed_data", config=config)[0]
    image = image[0, :, :, 0]

# If the image data is missing, generate a dummy image
except KeyError:
    image = np.ones((128, 128))

# Apply log compression
image = log_compress(image/np.max(image))

fig, ax = plt.subplots(figsize=(6, 6))
ax.imshow(image,
          cmap='gray',
          extent=[scan.xlims[0], scan.xlims[1], scan.zlims[1], scan.zlims[0]])

# Add labels
ax.set_xlabel('Lateral distance [m]')
ax.set_ylabel('Depth [m]')

# Turn the figure black
ax.set_facecolor((0, 0, 0))
fig.patch.set_facecolor((0, 0, 0))

# Turn the ticks white
ax.tick_params(axis='both', colors='white')

# Turn the labels white
ax.xaxis.label.set_color('white')
ax.yaxis.label.set_color('white')